In [194]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
import glob
import random
from collections import Counter

In [ ]:
def random_partition(arr,prob):
    "Particiona um array em duas partes aleatoriamente dado um probabilidade"
    result = [[],[]]
    for data in arr:
        result[0 if random.random() < prob else 1].append(data)
    return result


def manhattanDistance(X,Y):
    return np.sum(np.abs(X - Y),axis=1)

def euclideanDistance(X,Y):
    return np.sqrt(np.sum(np.square(X - Y), axis = 1))

In [202]:
class NearestNeighbor:
    def __init__(self):
        pass

    def train(self,X,y):
        """ X is N x D where each row is an example. Y is 1-dimension of size N """
        # the nearest neighbor classifier simply remembers all the training data
        self.Xtr = X
        self.ytr = y

    def predict(self,X, fnDistance = manhattanDistance, k=1):
        """ X is N x D where each row is an example we wish to predict label for """
        num_test = len(X)

        Ypred = np.zeros(num_test,dtype=self.ytr.dtype)

        for i in range(num_test):
            
            distances = np.sum(np.abs(self.Xtr - X[i,:]), axis=(2,1))
            
            """ min_index = np.argmin(distances)

            Ypred[i] = self.ytr[min_index]  """

            distance_sorted = sorted(zip(distances,self.ytr), key=lambda item: item[0])
            _,labels = zip(*distance_sorted)

            Ypred[i] = self.count_tuple(labels[:k])
            
        return Ypred

    def count_tuple(self,labels):
        ocorrence = Counter(labels)
        winner,winner_count = ocorrence.most_common(1)[0]
        num_winners = len([count for count in ocorrence.values() if count == winner_count])

        if num_winners == 1:
            return winner
        else: 
            return self.count_tuple(labels[:-1])



In [115]:
img = cv.imread("shapes/circle/0.png",cv.IMREAD_GRAYSCALE)

data = [] #[(img_data,label)]
max_shape_count = 100
for shape in glob.glob('shapes/*'):
    count = 0
    for fn in glob.glob(f'{shape}/*'):
        img = cv.imread(fn,cv.IMREAD_GRAYSCALE)
        img_shape = shape.split('/')[1]
        data.append([img,img_shape])
        count += 1
        if count >= max_shape_count:
            break


train_data, test_data = random_partition(data,0.7)

train_data_X, train_data_y = zip(*train_data)
test_data_X, test_data_y = zip(*test_data)

train_data_X = np.array(train_data_X)
train_data_y = np.array(train_data_y)
test_data_X = np.array(test_data_X)
test_data_y = np.array(test_data_y)


In [206]:
knn = NearestNeighbor()

knn.train(train_data_X, train_data_y)

for k in [1,3,5,7,9,11,13,15,17,19]:

    Y_pred_man = knn.predict(test_data_X,fnDistance=manhattanDistance,k=k)

    count = 0
    for i,_ in enumerate(test_data_y):
        if Y_pred_man[i] == test_data_y[i]:
            count += 1

    print(f'k={k} - P={count/len(test_data_y)}')


k=1 - P=0.5045871559633027
k=3 - P=0.45871559633027525
k=5 - P=0.4036697247706422
k=7 - P=0.4036697247706422
k=9 - P=0.3761467889908257
k=11 - P=0.3577981651376147
k=13 - P=0.3577981651376147
k=15 - P=0.3486238532110092
k=17 - P=0.3394495412844037
k=19 - P=0.3119266055045872


In [207]:
for k in [1,3,5,7,9,11,13,15,17,19]:

    Y_pred_man = knn.predict(test_data_X,fnDistance=euclideanDistance,k=k)

    count = 0
    for i,_ in enumerate(test_data_y):
        if Y_pred_man[i] == test_data_y[i]:
            count += 1

    print(f'k={k} - P={count/len(test_data_y)}')

k=1 - P=0.5045871559633027
k=3 - P=0.45871559633027525
k=5 - P=0.4036697247706422
k=7 - P=0.4036697247706422
k=9 - P=0.3761467889908257
k=11 - P=0.3577981651376147
k=13 - P=0.3577981651376147
k=15 - P=0.3486238532110092
k=17 - P=0.3394495412844037
k=19 - P=0.3119266055045872
